In [15]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, BertForSequenceClassification
from data.acronymDataset import AcronymDataset
from evaluate import load
import numpy as np
import torch

In [16]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=config.max_position_embeddings)
pre_trained_model = AutoModel.from_pretrained(model_name).to('mps')

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test with the data

In [17]:
# %load_ext autoreload
# %autoreload 2
torch.manual_seed(5)
file_path = 'data/acronym_data.txt'
dataset = AcronymDataset(file_path=file_path, tokenizer=tokenizer)
data = dataset.data

[INFO] Dataset already been loaded, using the cached dataset..


In [18]:
dataset.preprocss_dataset()

In [19]:
train_loader, val_loader = dataset.get_dataloaders(train_size=0.9, batch_size=32)

In [20]:
batch = next(iter(train_loader)).to('mps')

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [21]:
# %load_ext autoreload
# %autoreload 2
from models.multiHeadModel import MultiHeadModel
from models.heads import ClassificationHead

In [12]:
in_features = config.hidden_size
two_labels_head = ClassificationHead(in_features=in_features, out_features=2).to('mps')
four_labels_head = ClassificationHead(in_features=in_features, out_features=4).to('mps')

classifiers = {
    "two_labels_head": two_labels_head,
    "four_labels_head": four_labels_head
}

In [13]:
multi_head_model = MultiHeadModel(pre_trained_model, classifiers)

In [ ]:

with torch.no_grad():
    output = multi_head_model(batch, "two_labels_head")

output

In [41]:
metric = load("accuracy")
labels = batch['labels']
predictions = np.argmax(output.cpu().numpy(), axis=-1)
res = metric.compute(predictions=predictions, references=labels)

res

{'accuracy': 0.34375}

In [ ]:
%load_ext autoreload
%autoreload 2

from utils.train import train
train_loader1, _ = dataset.get_dataloaders(train_size=0.9, batch_size=16)
train_loader2, _ = dataset.get_dataloaders(train_size=0.9, batch_size=32)

train_args = {
    "epochs": 1
}

heads_props = {
    "two_labels_head": {
        "train_loader": train_loader1
    },
    "four_labels_head": {
        "train_loader": train_loader2
    }
}

train(multi_head_model, heads_props, train_args)